In [18]:
from pandas import read_csv
from itertools import groupby
from operator import itemgetter

In [19]:
def consecutive_cap(tweetWordList_cappos):
    output=[]
    #identifies consequtive numbers in the sequence
    for k, g in groupby(enumerate(tweetWordList_cappos), lambda (i, x): i-x):
        output.append(map(itemgetter(1), g))
    return output

In [20]:
#%%timeit -o
def f(x,tweetWordList):
    '''phrase=""
    for word in x[:-1]:
        phrase+=tweetWordList[word]+" "
    phrase+=tweetWordList[x[-1]]'''
    
    list1=map(lambda word: tweetWordList[word]+" " , x[:-1])
    phrase="".join(list1)+tweetWordList[x[-1]]
    return phrase

In [21]:
df = read_csv('/home/satadisha/Desktop/GitProjects/ELTweetTracker/Collection of Tweets.csv', index_col='ID', header=0)
#print df.columns.values.tolist()

In [23]:
#%%timeit -o
count=0
for index, row in df.iterrows():
    if count<1:
        tweetText=row['Tweets']
        #capitalization module
        #if all words are capitalized:
        if tweetText.isupper():
            print "All caps module"
        else:
            #returns word list split at whitespaces; retains punctuation
            tweetWordList=tweetText.split()
            #print [j for (i, j) in enumerate(tweetWordList)]
            
            #returns position of words that are capitalized
            tweetWordList_cappos = map(lambda (index, word) : index, filter(lambda (index, word) : word[0].isupper(), enumerate(tweetWordList)))
            #print tweetWordList_cappos
            output= consecutive_cap(tweetWordList_cappos)
            
            #consecutive capitalized phrases
            consecutive_cap_phrases=map(lambda x: f(x,tweetWordList), output)
            print consecutive_cap_phrases
            
            
            '''#implement the punctuation clause'''
            
        count+=1
    else:
        break

['U.S. Presidential Election', 'Trump, Clinton', 'Rubio...']
